3. Для всех инструментов, входящих в портфель, реализовать оценку их справедливой стоимости в зависимости от риск-факторов. Критически обсудить выбор модели. Обязательно проверить точность модели.

В качестве модели будем строить [ordinary least squares](https://en.wikipedia.org/wiki/Ordinary_least_squares#:~:text=Further%20reading-,Linear%20model,-%5Bedit%5D).

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
risks = pd.read_csv('risks.csv', skiprows = [1])
risks.set_index(risks.columns[0], inplace=True)
risks.rename_axis(None, inplace=True)

# Добавляем константу
risks = sm.add_constant(risks)

C:\Users\wchhi\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [3]:
risks.head()

,const,RTSI_RUB,RTSI_USD,IMOEX,CNY,USD,ZeroCoupon
2016-01-06,1.0,-0.011840,-0.021097,-0.003170,0.006311,0.019696,-0.0004
2016-01-11,1.0,-0.031959,-0.051152,-0.037762,0.042947,0.019409,-0.0001
2016-01-12,1.0,0.002648,-0.005450,0.004612,0.011449,0.011206,-0.0009
2016-01-13,1.0,-0.000440,-0.002402,-0.003266,-0.007223,-0.005035,0.0174
2016-01-14,1.0,0.000272,-0.000951,-0.002380,-0.010295,-0.008164,-0.0007


In [4]:
idx = pd.date_range("2016-01-05", "2021-12-30")

#### Облигации

В облигации будет оцениваться доходность по средневзвешенной цене, % годовых

In [67]:
bonds = [
    "SU26207RMFS9",
    "SU26209RMFS5",
    "SU26211RMFS1",
    "SU26212RMFS9",
    "SU26215RMFS2"
]

bonds_y = []

In [68]:
for i, bond in enumerate(bonds):
    if i in [0, 3, 2, 4]:
        bond_i = pd.read_csv("..\\data\\bonds\\history\\" + bond + ".csv", index_col = 'TRADEDATE',
                            skiprows = [1])['YIELDATWAP']
    else:
        bond_i = pd.read_csv("..\\data\\bonds\\history\\" + bond + ".csv", index_col = 'TRADEDATE')['YIELDATWAP']
        
    bond_i = bond_i.pct_change().dropna()
    bond_i.index = pd.to_datetime(bond_i.index)
    bond_i.reindex(idx).interpolate(inplace=True)
    
    bonds_y.append(bond_i)

Для каждого из бондов строим модель, смотрим результат:

In [69]:
reses_bonds = []

In [70]:
for i, _ in enumerate(bonds_y):
    model = sm.OLS(bonds_y[i].values, risks, missing='drop')
    res = model.fit()
    reses_bonds.append(res.summary())

C:\Users\wchhi\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1666: RuntimeWarning: invalid value encountered in subtract
  return np.sum(weights * (model.endog - mean)**2)


In [73]:
reses_bonds[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.225
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     72.82
Date:                Mon, 25 Apr 2022   Prob (F-statistic):           7.15e-80
Time:                        03:48:38   Log-Likelihood:                 4900.0
No. Observations:                1513   AIC:                            -9786.
Df Residuals:                    1506   BIC:                            -9749.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.135e-05      0.000      0.169      0.866      -0.000       0.001
RTSI_RUB       0.4163      0.042      9.941      0.000       0.334       0.498
RTSI_USD      -0.4829      0.054     -8.964      0.000      -0.589      -0.377
IMOEX         -0.1241      0.073     -1.699      0.090      -0.267       0.019
CNY            0.1075      0.064      1.675      0.094      -0.018       0.233
USD           -0.0732      0.073     -1.001      0.317      -0.217       0.070
ZeroCoupon     0.0277      0.054      0.517      0.606      -0.077       0.133
==============================================================================
Omnibus:                      628.141   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19809.877
Skew:                           1.298   Prob(JB):                         0.00
Kurtosis:                      20.535   Cond. No.                         406.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

$R^2 \in [0.225;0.380]$

**Акции и валюта**

Строим модель динамики для каждой акции, в качестве **справедливой стоимости** выберем цену [weighted average price](https://www.foxbusiness.com/markets/how-to-calculate-weighted-average-price-per-share#:~:text=In%20order%20to%20calculate%20your%20weighted%20average%20price%20per%20share%2C%20simply%20multiply%20each%20purchase%20price%20by%20the%20amount%20of%20shares%20purchased%20at%20that%20price%2C%20add%20them%20together%2C%20and%20then%20divide%20by%20the%20total%20number%20of%20shares.), так как для долговременного прогноза кажется наименее пере- или недооценённым использовать взвешенную стоимость акции по всем торгам за прошедшее время.

In [10]:
stocks = [
    "ALRS",
    "GAZP",
    "GMKN",
    "LKOH",
    "MGNT",
    "NLMK",
    "NVTK",
    "PLZL",
    "POLY",
    "ROSN"
]

stocks_y = []

In [11]:
for stock in stocks:
    stock_i = pd.read_csv("..\\data\\stocks\\history\\" + stock + ".csv", index_col = 'TRADEDATE', 
                          skiprows=[1])['WAPRICE']
    
    stock_i = stock_i.pct_change().dropna()
    stock_i.index = pd.to_datetime(stock_i.index)
    stock_i.reindex(idx).interpolate(inplace=True)
    
    stocks_y.append(stock_i)

Валюта

In [16]:
currs = [
    "CNY000000TOD",
    "USD000000TOD"
]

currs_y = []

In [17]:
for curr in currs:
    curr_i = pd.read_csv("..\\data\\currencies\\history\\" + curr + ".csv", index_col = 'TRADEDATE', 
                          skiprows=[1])['WAPRICE']
    
    curr_i = curr_i.pct_change().dropna()
    curr_i.index = pd.to_datetime(curr_i.index)
    curr_i.reindex(idx).interpolate(inplace=True)
    
    currs_y.append(curr_i)

Расчёт результатов для акций и валют

In [21]:
reses_stocks = []

In [22]:
for i, _ in enumerate(stocks_y):
    model = sm.OLS(stocks_y[i].values, risks)
    res = model.fit()
    reses_stocks.append(res.summary())

In [74]:
reses_stocks

[<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.183
 Model:                            OLS   Adj. R-squared:                  0.180
 Method:                 Least Squares   F-statistic:                     56.15
 Date:                Mon, 25 Apr 2022   Prob (F-statistic):           9.09e-63
 Time:                        03:16:04   Log-Likelihood:                 4293.8
 No. Observations:                1513   AIC:                            -8574.
 Df Residuals:                    1506   BIC:                            -8536.
 Df Model:                           6                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------

$R^2 \in [0.037;0.469]$

In [34]:
reses_currs = []

In [35]:
for i, _ in enumerate(currs_y):
    model = sm.OLS(currs_y[i].values, risks)
    res = model.fit()
    reses_currs.append(res.summary())

$R^2 \in [0.541;0.785]$

In [75]:
reses_currs

[<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.541
 Model:                            OLS   Adj. R-squared:                  0.539
 Method:                 Least Squares   F-statistic:                     296.2
 Date:                Mon, 25 Apr 2022   Prob (F-statistic):          1.15e-250
 Time:                        03:17:27   Log-Likelihood:                 5671.8
 No. Observations:                1513   AIC:                        -1.133e+04
 Df Residuals:                    1506   BIC:                        -1.129e+04
 Df Model:                           6                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------